# Part 2: Practical Implementation

## Task 1: Edge AI Prototype

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import time
import os

In [4]:
print("TensorFlow version:", tf.__version__)

# Set random seed for reproducible results
tf.random.set_seed(42)
np.random.seed(42)

print("Creating simple dataset...")

TensorFlow version: 2.19.0
Creating simple dataset...


## Create Simple Dataset

In [5]:
# 5 types of recyclable items
class_names = ['plastic', 'paper', 'glass', 'metal', 'organic']
num_classes = len(class_names)

# Create simple synthetic data
def create_simple_dataset():
    # Small dataset for quick training
    samples_per_class = 50  # Small for quick demo
    img_size = 64  # Smaller images for faster processing
    
    # Training data
    X_train = []
    y_train = []
    
    # Test data  
    X_test = []
    y_test = []
    
    for class_id, class_name in enumerate(class_names):
        # Training samples
        for i in range(samples_per_class):
            # Create unique pattern for each class
            img = np.random.rand(img_size, img_size, 3)
            if class_name == 'plastic':
                img[:, :, 0] += 0.5  # More red
            elif class_name == 'paper':
                img[:, :, 1] += 0.5  # More green  
            elif class_name == 'glass':
                img[:, :, 2] += 0.5  # More blue
            elif class_name == 'metal':
                img += 0.3  # Brighter
            else:  # organic
                img[:, :, 0] += 0.3
                img[:, :, 1] += 0.3
            
            img = np.clip(img, 0, 1)
            X_train.append(img)
            y_train.append(class_id)
        
        # Test samples (20% of training)
        for i in range(samples_per_class // 5):
            img = np.random.rand(img_size, img_size, 3)
            if class_name == 'plastic':
                img[:, :, 0] += 0.5
            elif class_name == 'paper':
                img[:, :, 1] += 0.5
            elif class_name == 'glass':
                img[:, :, 2] += 0.5
            elif class_name == 'metal':
                img += 0.3
            else:  # organic
                img[:, :, 0] += 0.3
                img[:, :, 1] += 0.3
            
            img = np.clip(img, 0, 1)
            X_test.append(img)
            y_test.append(class_id)
    
    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)

# Create dataset
X_train, y_train, X_test, y_test = create_simple_dataset()

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Image size: {X_train[0].shape}")
print(f"Classes: {class_names}")

Training samples: 250
Test samples: 50
Image size: (64, 64, 3)
Classes: ['plastic', 'paper', 'glass', 'metal', 'organic']


## Build Simple Model

In [6]:
print("\nBuilding simple model...")

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=X_train[0].shape),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model created!")
print(f"Model parameters: {model.count_params():,}")



Building simple model...


C:\Users\USER\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model created!
Model parameters: 822,597


## Train Model

In [7]:
print("\nTraining model...")

history = model.fit(
    X_train, y_train,
    epochs=20,  # Quick training
    validation_data=(X_test, y_test),
    verbose=1
)




Training model...
Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.2143 - loss: 1.8397 - val_accuracy: 0.4000 - val_loss: 1.3480
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.4145 - loss: 1.2286 - val_accuracy: 0.8000 - val_loss: 0.7559
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.8034 - loss: 0.6548 - val_accuracy: 1.0000 - val_loss: 0.2643
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 1.0000 - loss: 0.2383 - val_accuracy: 1.0000 - val_loss: 0.0678
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 1.0000 - loss: 0.0459 - val_accuracy: 1.0000 - val_loss: 0.0104
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 1.0000 - loss: 0.0081 - val_accuracy: 1.0000 - val_loss: 0.0024
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 9.9125e-04
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 1.0000 - loss: 8.5268e-04 - val_accuracy

## Evaluate Model

In [8]:

print("\nEvaluating model...")

# Test accuracy
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.3f}")

# Predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, predicted_classes, target_names=class_names))




Evaluating model...
Test Accuracy: 1.000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step

Classification Report:
              precision    recall  f1-score   support

     plastic       1.00      1.00      1.00        10
       paper       1.00      1.00      1.00        10
       glass       1.00      1.00      1.00        10
       metal       1.00      1.00      1.00        10
     organic       1.00      1.00      1.00        10

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50



## Convert to TensorFlow Lite

In [9]:
print("\nConverting to TensorFlow Lite...")

# Convert model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save TensorFlow Lite model
tflite_path = 'recyclable_classifier.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"TensorFlow Lite model saved: {tflite_path}")
print(f"Model size: {len(tflite_model) / 1024:.1f} KB")




Converting to TensorFlow Lite...
INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmpovwo14a3\assets


INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmpovwo14a3\assets


Saved artifact at 'C:\Users\USER\AppData\Local\Temp\tmpovwo14a3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2431128962144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2431128532976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2431128968480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2431128967072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2431128964608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2431128962320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2431128969712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2431128969008: TensorSpec(shape=(), dtype=tf.resource, name=None)
TensorFlow Lite model saved: recyclable_classifier.tflite
Model size: 812.0 KB


## Test TensorFlow Lite Model

In [10]:
print("\nTesting TensorFlow Lite model...")

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test on few samples
correct = 0
total = 0

for i in range(min(20, len(X_test))):
    # Prepare input
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    
    # Run inference
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    
    # Get result
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted = np.argmax(output_data)
    
    if predicted == y_test[i]:
        correct += 1
    total += 1

tflite_accuracy = correct / total
print(f"TensorFlow Lite Accuracy: {tflite_accuracy:.3f}")




Testing TensorFlow Lite model...
TensorFlow Lite Accuracy: 1.000


C:\Users\USER\anaconda3\envs\tf_env\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


## Measure Speed

In [11]:
print("\nMeasuring inference speed...")

# Test original model speed
start_time = time.time()
_ = model.predict(X_test[:10])
original_time = (time.time() - start_time) / 10

# Test TensorFlow Lite speed
start_time = time.time()
for i in range(10):
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    _ = interpreter.get_tensor(output_details[0]['index'])
tflite_time = (time.time() - start_time) / 10

print(f"Original model: {original_time*1000:.1f} ms per image")
print(f"TensorFlow Lite: {tflite_time*1000:.1f} ms per image")
print(f"Speed improvement: {original_time/tflite_time:.1f}x faster")




Measuring inference speed...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Original model: 20.7 ms per image
TensorFlow Lite: 1.9 ms per image
Speed improvement: 11.1x faster


## Simple Report

In [12]:

print("\n" + "="*50)
print("EDGE AI RECYCLABLE CLASSIFIER - RESULTS")
print("="*50)
print(f"Model Accuracy: {test_accuracy:.1%}")
print(f"TensorFlow Lite Accuracy: {tflite_accuracy:.1%}")
print(f"Model Size: {len(tflite_model)/1024:.1f} KB")
print(f"Inference Speed: {tflite_time*1000:.1f} ms per image")
print(f"Classes: {', '.join(class_names)}")
print("="*50)




EDGE AI RECYCLABLE CLASSIFIER - RESULTS
Model Accuracy: 100.0%
TensorFlow Lite Accuracy: 100.0%
Model Size: 812.0 KB
Inference Speed: 1.9 ms per image
Classes: plastic, paper, glass, metal, organic


## Edge AI Benefits

In [13]:
print("\nEDGE AI BENEFITS FOR REAL-TIME APPLICATIONS:")
print("1. FAST: Only", f"{tflite_time*1000:.1f} ms", "per classification")
print("2. SMALL: Only", f"{len(tflite_model)/1024:.1f} KB", "model size")
print("3. OFFLINE: Works without internet")
print("4. PRIVATE: Data stays on device")
print("5. CHEAP: No cloud costs")




EDGE AI BENEFITS FOR REAL-TIME APPLICATIONS:
1. FAST: Only 1.9 ms per classification
2. SMALL: Only 812.0 KB model size
3. OFFLINE: Works without internet
4. PRIVATE: Data stays on device
5. CHEAP: No cloud costs


## Simple Deployment Code

In [14]:
deployment_code = '''
# Simple Raspberry Pi Deployment Code
# Save as: raspberry_pi_deploy.py

import tensorflow as tf
import numpy as np
import cv2

# Load the model
interpreter = tf.lite.Interpreter(model_path='recyclable_classifier.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

class_names = ['plastic', 'paper', 'glass', 'metal', 'organic']

def classify_image(image_path):
    # Load and preprocess image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 64))
    image = image.astype(np.float32) / 255.0
    image = np.expand_dims(image, axis=0)
    
    # Run inference
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    
    # Get result
    output = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output)
    confidence = np.max(output)
    
    return class_names[predicted_class], confidence

# Example usage:
# result, confidence = classify_image('recyclable_item.jpg')
# print(f"This is {result} with {confidence:.2f} confidence")
'''

with open('raspberry_pi_deploy.py', 'w') as f:
    f.write(deployment_code)

print("\nDeployment code saved: raspberry_pi_deploy.py")


Deployment code saved: raspberry_pi_deploy.py


### Show Sample Results

In [ ]:
print("\nSample Predictions:")
for i in range(5):
    actual = class_names[y_test[i]]
    predicted = class_names[predicted_classes[i]]
    confidence = np.max(predictions[i])
    print(f"Sample {i+1}: Actual={actual}, Predicted={predicted}, Confidence={confidence:.2f}")

print("\n✅ ALL DONE! Your Edge AI model is ready for deployment!")